

# Item based filtering
This approach is mostly preferred since the movie don't change much. We can rerun this model once a week unlike User based where we have to frequently run the model.

In this kernel, We look at the implementation of Item based filtering

In [1]:
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
!pip install streamlit


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
movies = pd.read_csv("data/movies.csv")
ratings = pd.read_csv("data/ratings.csv")
tags = pd.read_csv("data/tags.csv")

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [6]:
no_user_voted = ratings.groupby('movieId')['rating'].agg('count')
no_movies_voted = ratings.groupby('userId')['rating'].agg('count')

movie_user_matrix = ratings.pivot(index='movieId',columns='userId',values='rating').fillna(0)
movie_user_matrix = movie_user_matrix.fillna(0)
movie_user_matrix = movie_user_matrix.loc[:,no_movies_voted[no_movies_voted > 50].index]
movie_user_matrix = movie_user_matrix.loc[no_user_voted[no_user_voted > 10].index,:]
movie_user_matrix

userId,1,4,6,7,10,11,15,16,17,18,...,600,601,602,603,604,605,606,607,608,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5,3.5,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0,...,0.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
176371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
177765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
filtroUserId = movie_user_matrix.columns.tolist()
filtroMovieId = movie_user_matrix.index.tolist()

In [8]:
# ratings = ratings[ratings['userId'].isin(filtroUserId)]
# ratings = ratings[ratings['movieId'].isin(filtroMovieId)]

# movies = movies[movies['movieId'].isin(filtroMovieId)]

In [9]:
userId_movieId_rating_genres = pd.merge(ratings, movies, left_on='movieId', right_on='movieId', how='inner')
userId_movieId_rating_genres = userId_movieId_rating_genres[userId_movieId_rating_genres['userId'].isin(filtroUserId)]
userId_movieId_rating_genres = userId_movieId_rating_genres[userId_movieId_rating_genres['movieId'].isin(filtroMovieId)]
userId_movieId_rating_genres.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [10]:
userId_movieId_rating_genres['genresSplit'] = userId_movieId_rating_genres['genres'].str.split('|')

userId_rating_genres = userId_movieId_rating_genres[['userId', 'genresSplit', 'rating']]
userId_rating_genres = userId_rating_genres.explode('genresSplit')
userId_rating_genres.head()

,userId,genresSplit,rating
0,1,Adventure,4.0
0,1,Animation,4.0
0,1,Children,4.0
0,1,Comedy,4.0
0,1,Fantasy,4.0


In [11]:
genres_count_mean = userId_rating_genres\
    .groupby('genresSplit')\
    .agg(
        countGenres=('genresSplit', 'count'),
        ratingMean=('rating', 'mean'),
    ).reset_index()

genres_count_mean['porcGenres'] = genres_count_mean['countGenres'] / genres_count_mean['countGenres'].sum() * 100
genres_count_mean['rank'] = genres_count_mean['porcGenres'].transform(lambda x: x.rank(ascending=False))
genres_count_mean = genres_count_mean.sort_values('porcGenres', ascending=True)
mainListGenres = genres_count_mean[genres_count_mean['rank']<=10]['genresSplit'].tolist()
mainListGenres

['Children',
 'Fantasy',
 'Crime',
 'Romance',
 'Sci-Fi',
 'Adventure',
 'Thriller',
 'Action',
 'Comedy',
 'Drama']

In [12]:
userId_rating_genres_mainGenres = userId_rating_genres[userId_rating_genres['genresSplit'].isin(mainListGenres)]

genres_userId_matrix = userId_rating_genres_mainGenres\
    .groupby(['userId', 'genresSplit'])\
        .agg(ratingMean=('rating', 'mean'))\
            .reset_index()\
                .pivot(index='genresSplit', columns='userId', values='ratingMean')\
                    .fillna(0)

In [13]:
genres_userId_matrix.shape

(10, 378)

In [14]:
genres_userId_matrix

userId,1,4,6,7,10,11,15,16,17,18,...,600,601,602,603,604,605,606,607,608,610
genresSplit,,,,,,,,,,,,,,,,,,,,,
Action,4.309524,3.304348,3.672727,3.295082,3.458333,3.522727,3.215517,3.520000,4.230769,3.578378,...,2.676000,4.347222,3.560976,3.150794,3.600000,3.320000,3.238095,3.724638,3.357708,3.722741
Adventure,4.410256,3.750000,4.027027,3.339623,3.685185,3.555556,3.339286,3.775000,4.285714,3.634328,...,3.211111,4.437500,3.206897,2.942529,3.666667,3.375000,3.500000,3.441860,3.260116,3.806452
Children,4.617647,4.111111,3.781250,3.200000,3.576923,0.000000,2.690476,4.000000,4.214286,3.529412,...,3.011628,4.555556,3.000000,3.142857,3.666667,3.219512,3.500000,3.312500,2.525000,3.825000
Comedy,4.307692,3.576471,3.445652,3.238636,3.245455,3.400000,3.351852,3.750000,4.157895,3.432540,...,3.101695,4.388889,3.270270,3.299065,3.250000,3.369565,3.515625,3.333333,2.824104,3.787402
Crime,4.375000,3.807692,3.281250,3.625000,3.115385,3.750000,3.578947,3.750000,4.250000,3.925234,...,2.915493,4.558824,3.730769,3.764706,3.705882,3.150000,3.798851,3.814815,3.619231,3.933333
Drama,4.508197,3.510638,3.770833,3.229167,3.106383,4.181818,3.764706,3.723404,4.186275,3.912791,...,2.948347,4.425532,3.344262,3.633094,3.545455,3.256757,3.823529,4.014085,3.428571,3.992806
Fantasy,4.333333,3.875000,3.700000,3.400000,3.576923,2.000000,2.906250,3.937500,4.323529,3.644231,...,3.514019,4.526316,3.714286,3.405405,3.454545,3.265957,3.611842,3.500000,3.052381,3.730000
Romance,4.285714,3.300000,3.767857,2.788462,3.415094,4.230769,3.916667,3.653846,3.928571,3.666667,...,2.960145,4.400000,3.217391,3.467742,3.071429,3.406977,3.731183,3.535714,2.884211,3.767442
Sci-Fi,4.230769,2.833333,3.526316,3.146341,2.000000,3.500000,3.586207,3.687500,4.400000,3.642857,...,2.932584,4.300000,3.333333,3.164179,3.533333,3.309524,3.564286,3.250000,3.337580,3.769886


In [15]:
genres_userId_matrix.index

Index(['Action', 'Adventure', 'Children', 'Comedy', 'Crime', 'Drama',
       'Fantasy', 'Romance', 'Sci-Fi', 'Thriller'],
      dtype='object', name='genresSplit')

In [34]:
data = {'idNewUser': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

indexNewUser = [
    'Action', 'Adventure', 'Children', 'Comedy', 'Crime', 
    'Drama', 'Fantasy', 'Romance', 'Sci-Fi', 'Thriller']

user_movie_matrix_newUser = pd.DataFrame(data, index=indexNewUser)
user_movie_matrix_newUser

,idNewUser
Action,0
Adventure,0
Children,0
Comedy,0
Crime,0
Drama,0
Fantasy,0
Romance,0
Sci-Fi,0
Thriller,0


In [35]:
for it in range(5, 10):
    print(it)

5
6
7
8
9


In [36]:
genres_userId_matrix_Predict = pd.concat([user_movie_matrix_newUser, genres_userId_matrix], axis=1)
genres_userId_matrix_Predict

,idNewUser,1,4,6,7,10,11,15,16,17,...,600,601,602,603,604,605,606,607,608,610
Action,0,4.309524,3.304348,3.672727,3.295082,3.458333,3.522727,3.215517,3.520000,4.230769,...,2.676000,4.347222,3.560976,3.150794,3.600000,3.320000,3.238095,3.724638,3.357708,3.722741
Adventure,0,4.410256,3.750000,4.027027,3.339623,3.685185,3.555556,3.339286,3.775000,4.285714,...,3.211111,4.437500,3.206897,2.942529,3.666667,3.375000,3.500000,3.441860,3.260116,3.806452
Children,0,4.617647,4.111111,3.781250,3.200000,3.576923,0.000000,2.690476,4.000000,4.214286,...,3.011628,4.555556,3.000000,3.142857,3.666667,3.219512,3.500000,3.312500,2.525000,3.825000
Comedy,0,4.307692,3.576471,3.445652,3.238636,3.245455,3.400000,3.351852,3.750000,4.157895,...,3.101695,4.388889,3.270270,3.299065,3.250000,3.369565,3.515625,3.333333,2.824104,3.787402
Crime,0,4.375000,3.807692,3.281250,3.625000,3.115385,3.750000,3.578947,3.750000,4.250000,...,2.915493,4.558824,3.730769,3.764706,3.705882,3.150000,3.798851,3.814815,3.619231,3.933333
Drama,0,4.508197,3.510638,3.770833,3.229167,3.106383,4.181818,3.764706,3.723404,4.186275,...,2.948347,4.425532,3.344262,3.633094,3.545455,3.256757,3.823529,4.014085,3.428571,3.992806
Fantasy,0,4.333333,3.875000,3.700000,3.400000,3.576923,2.000000,2.906250,3.937500,4.323529,...,3.514019,4.526316,3.714286,3.405405,3.454545,3.265957,3.611842,3.500000,3.052381,3.730000
Romance,0,4.285714,3.300000,3.767857,2.788462,3.415094,4.230769,3.916667,3.653846,3.928571,...,2.960145,4.400000,3.217391,3.467742,3.071429,3.406977,3.731183,3.535714,2.884211,3.767442
Sci-Fi,0,4.230769,2.833333,3.526316,3.146341,2.000000,3.500000,3.586207,3.687500,4.400000,...,2.932584,4.300000,3.333333,3.164179,3.533333,3.309524,3.564286,3.250000,3.337580,3.769886
Thriller,0,4.192308,3.515152,3.589286,3.434211,3.000000,3.656250,3.464286,3.785714,4.277778,...,2.752101,4.400000,3.723404,3.561644,3.760000,3.000000,3.561069,4.107143,3.552174,3.780242


In [37]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

In [38]:
user_similarity = cosine_similarity(genres_userId_matrix_Predict.T)
user_similarity_df = pd.DataFrame(user_similarity, index=genres_userId_matrix_Predict.columns, columns=genres_userId_matrix_Predict.columns)

In [39]:
user_similarity_df

,idNewUser,1,4,6,7,10,11,15,16,17,...,600,601,602,603,604,605,606,607,608,610
idNewUser,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,1.000000,0.996788,0.998737,0.997648,0.991293,0.928860,0.992972,0.999501,0.999141,...,0.997264,0.999758,0.996017,0.996722,0.998283,0.999062,0.998765,0.996259,0.992948,0.999703
4,0.0,0.996788,1.000000,0.994869,0.996046,0.995359,0.909457,0.983772,0.997303,0.994965,...,0.995875,0.996875,0.992612,0.993280,0.995365,0.993934,0.994808,0.992546,0.986545,0.995690
6,0.0,0.998737,0.994869,1.000000,0.995246,0.992255,0.930280,0.992142,0.998238,0.997943,...,0.996896,0.998360,0.994089,0.993734,0.996819,0.998632,0.997036,0.994946,0.991448,0.998161
7,0.0,0.997648,0.996046,0.995246,1.000000,0.988343,0.931140,0.990458,0.997928,0.998757,...,0.995517,0.998336,0.998430,0.996494,0.999082,0.995981,0.996893,0.996808,0.996176,0.998112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,0.0,0.999062,0.993934,0.998632,0.995981,0.990165,0.936014,0.994399,0.998407,0.998615,...,0.997450,0.998988,0.995407,0.995362,0.996277,1.000000,0.998070,0.994488,0.992248,0.998949
606,0.0,0.998765,0.994808,0.997036,0.996893,0.987705,0.938587,0.996114,0.998692,0.998237,...,0.996862,0.999098,0.996730,0.998792,0.996798,0.998070,1.000000,0.996971,0.994445,0.999424
607,0.0,0.996259,0.992546,0.994946,0.996808,0.987206,0.944660,0.994556,0.995678,0.996361,...,0.991441,0.996810,0.997744,0.998200,0.996948,0.994488,0.996971,1.000000,0.997135,0.997429
608,0.0,0.992948,0.986545,0.991448,0.996176,0.978577,0.954239,0.994440,0.992287,0.995336,...,0.988768,0.993937,0.997283,0.995116,0.995928,0.992248,0.994445,0.997135,1.000000,0.995029


In [40]:
def get_similar_users(user_similarity_df, newUser='idNewUser', n_recommend_users = 5):
    similar_users = user_similarity_df.loc[newUser]
    similar_users = similar_users[(similar_users > 0) & (similar_users < 1)].sort_values(ascending=False)

    return similar_users[:n_recommend_users].index.tolist()



userId2Recommend = get_similar_users(user_similarity_df, newUser='idNewUser', n_recommend_users = 5)
userId2Recommend

[]

In [41]:
# def get_most_prefered_movie_by_user(userIdSearh, movie_user_matrix, n_most_movies = 2):
#     try:
#         most_idMovies = movie_user_matrix.loc[userIdSearh][movie_user_matrix.loc[userIdSearh] > 0].sort_values(ascending=False)
#         return most_idMovies[:n_most_movies].index.tolist()
#     except KeyError:
#         return None

In [42]:
def get_most_prefered_movie_by_user(userIdSearhList, movie_user_matrix, n_most_movies = 2):
    most_idMoviesList = []
    for userIdSearh in userIdSearhList:
        try:
            most_idMovies = movie_user_matrix.loc[userIdSearh][movie_user_matrix.loc[userIdSearh] > 0].sort_values(ascending=False)
            most_idMovies = most_idMovies[:n_most_movies].index.tolist()
            most_idMoviesList += most_idMovies
        except KeyError:
            pass

    if len(most_idMoviesList)>0:
        return most_idMoviesList
    else:
        return []

In [43]:
movieId2Recommend = get_most_prefered_movie_by_user(userId2Recommend, movie_user_matrix, n_most_movies = 3)
movieId2Recommend

[]

In [44]:
csr_data = csr_matrix(movie_user_matrix.values)
movie_user_matrix_recommend = movie_user_matrix.reset_index()

In [45]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

In [46]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [47]:
# def get_movie_recommendation(movieIdRef, movie_user_matrix_recommend, n_movies_to_reccomend = 5):
#     # movie_list = movies[movies['title'].str.contains(movie_name)] 
#     movie_list = movies[movies['movieId'] == movieIdRef] 
#     if len(movie_list):        
#         movie_idx= movie_list.iloc[0]['movieId']
#         movie_idx = movie_user_matrix_recommend[movie_user_matrix_recommend['movieId'] == movie_idx].index[0]
        
#         distances , indices = knn.kneighbors(csr_data[movie_idx],n_neighbors=n_movies_to_reccomend+1)    
#         rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),\
#                                key=lambda x: x[1])[:0:-1]
        
#         recommend_frame = []
        
#         for val in rec_movie_indices:
#             movie_idx = movie_user_matrix_recommend.iloc[val[0]]['movieId']
#             idx = movies[movies['movieId'] == movie_idx].index
#             recommend_frame.append({'Title':movies.iloc[idx]['title'].values[0], 
#                                     'Genres':movies.iloc[idx]['genres'].values[0], 
#                                     'Distance':val[1]})
#         df = pd.DataFrame(recommend_frame,index=range(1,n_movies_to_reccomend+1))
#         return df
#     else:
#         return "No movies found. Please check your input"

In [48]:
def get_movie_recommendation(movieIdRef, movies, movie_user_matrix_recommend, n_movies_to_reccomend=5):
    # Filtrar la película basada en el movieIdRef
    movie_list = movies[movies['movieId'] == movieIdRef] 
    
    if len(movie_list) == 0:
        return "No movies found. Please check your input"
    
    # Obtener el movieId de la película seleccionada
    movie_idx = movie_list.iloc[0]['movieId']
    
    # Verificar si el movieId está en el movie_user_matrix_recommend
    user_movie_row = movie_user_matrix_recommend[movie_user_matrix_recommend['movieId'] == movie_idx]
    
    if user_movie_row.empty:
        return pd.DataFrame({'Title': [], 'Genres': [], 'Distance': []})
    
    # Obtener el índice de la película en la matriz
    movie_idx = user_movie_row.index[0]
    
    # Obtener las distancias y los índices de las películas recomendadas
    distances, indices = knn.kneighbors(csr_data[movie_idx], n_neighbors=n_movies_to_reccomend + 1)
    
    # Crear una lista de recomendaciones
    rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    
    recommend_frame = []
    
    for val in rec_movie_indices:
        recommended_movie_id = movie_user_matrix_recommend.iloc[val[0]]['movieId']
        idx = movies[movies['movieId'] == recommended_movie_id].index
        
        if len(idx) == 0:
            continue  # Si no se encuentra el índice, continuar con la siguiente iteración
        
        recommend_frame.append({
            'Title': movies.iloc[idx]['title'].values[0],
            'Genres': movies.iloc[idx]['genres'].values[0],
            'Distance': val[1]
        })
    
    # Crear un DataFrame para las recomendaciones
    df = pd.DataFrame(recommend_frame, index=range(1, len(recommend_frame) + 1))
    
    if df.empty:
        return "No recommendations found."
    
    return df

In [49]:
final_movie_recommendation = []
for recommendMovieId in movieId2Recommend:
    final_movie_recommendation.append(get_movie_recommendation(recommendMovieId, movies, movie_user_matrix_recommend, 3))

In [51]:
final_movie_recommendation

[]

In [50]:
pd.concat(final_movie_recommendation, axis=0)

ValueError: No objects to concatenate

In [34]:
import joblib

# Guardar los objetos en archivos usando joblib
joblib.dump(movies, 'pkl/movies.pkl')
joblib.dump(mainListGenres, 'pkl/mainListGenres.pkl')
joblib.dump(genres_userId_matrix, 'pkl/genres_userId_matrix.pkl')
joblib.dump(user_similarity_df, 'pkl/user_similarity_df.pkl')
joblib.dump(knn, 'pkl/knn.pkl')
joblib.dump(csr_data, 'pkl/csr_data.pkl')
joblib.dump(movie_user_matrix, 'pkl/movie_user_matrix.pkl')
joblib.dump(movie_user_matrix_recommend, 'pkl/movie_user_matrix_recommend.pkl')

print("Objetos guardados exitosamente.")

Objetos guardados exitosamente.


In [ ]:
import joblib

# Cargar los objetos desde los archivos
movies = joblib.load('pkl/movies.pkl')
mainListGenres = joblib.load('pkl/mainListGenres.pkl')
genres_userId_matrix = joblib.load('pkl/genres_userId_matrix.pkl')
user_similarity_df = joblib.load('pkl/user_similarity_df.pkl')
knn = joblib.load('pkl/knn.pkl')
csr_data = joblib.load('pkl/csr_data.pkl')
movie_user_matrix = joblib.load('pkl/movie_user_matrix.pkl')
movie_user_matrix_recommend = joblib.load('pkl/movie_user_matrix_recommend.pkl')

In [5]:
for i in range(0, 10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [1]:
import streamlit as st
import pandas as pd
import joblib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from  utils import get_similar_users, get_most_prefered_movie_by_user, get_movie_recommendation

In [2]:
movies = joblib.load('pkl/movies.pkl')
mainListGenres = joblib.load('pkl/mainListGenres.pkl')
genres_userId_matrix = joblib.load('pkl/genres_userId_matrix.pkl')
user_similarity_df = joblib.load('pkl/user_similarity_df.pkl')
knn = joblib.load('pkl/knn.pkl')
csr_data = joblib.load('pkl/csr_data.pkl')
movie_user_matrix = joblib.load('pkl/movie_user_matrix.pkl')
movie_user_matrix_recommend = joblib.load('pkl/movie_user_matrix_recommend.pkl')

In [3]:
params = [0,0,0,0,0,0,0,0,0,0]

In [23]:
data = {'idNewUser': params}
user_movie_matrix_newUser = pd.DataFrame(data, index=mainListGenres)

genres_userId_matrix_Predict = pd.concat([user_movie_matrix_newUser, genres_userId_matrix], axis=1)
userId2Recommend = get_similar_users(user_similarity_df, newUser='idNewUser', n_recommend_users = 10)
movieId2Recommend = get_most_prefered_movie_by_user(userId2Recommend, movie_user_matrix, n_most_movies = 5)

final_movie_recommendation = []
for recommendMovieId in movieId2Recommend:
    final_movie_recommendation.append(get_movie_recommendation(recommendMovieId, knn, csr_data, movies, movie_user_matrix_recommend, n_movies_to_reccomend=3))

dfRecommendationFinal = pd.concat(final_movie_recommendation, axis=0)
dfRecommendationFinal = dfRecommendationFinal.drop_duplicates(subset='Title', keep=False)
dfRecommendationFinal = dfRecommendationFinal.sort_values('Distance', ascending=False).reset_index()
dfRecommendationFinal = dfRecommendationFinal.drop('index', axis=1)

In [24]:

dfRecommendationFinal


,Title,Genres,Distance
0,Casino (1995),Crime|Drama,0.611757
1,Dead Man Walking (1995),Crime|Drama,0.591069
2,Heat (1995),Action|Crime|Thriller,0.589260
3,Carlito's Way (1993),Crime|Drama,0.554757
4,October Sky (1999),Drama,0.554722
5,"American President, The (1995)",Comedy|Drama|Romance,0.544679
6,"Man Without a Face, The (1993)",Drama,0.524218
7,Don Juan DeMarco (1995),Comedy|Drama|Romance,0.522566
8,"Piano, The (1993)",Drama|Romance,0.514590
9,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,0.447292
